In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/cleaned_posting_32556_salary_info.csv',header=0)

In [3]:
df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,min_salary,formatted_work_type,original_listed_time,expiry,formatted_experience_level,listed_time,work_type,currency,normalized_salary,zip_code
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,17.0,Full-time,2024-04-17 23:45:08,2024-05-17 23:45:08,NaN,2024-04-17 23:45:08,FULL_TIME,USD,38480.0,8540.0
1,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,45000.0,Full-time,2024-04-16 14:26:54,2024-05-16 14:26:54,NaN,2024-04-16 14:26:54,FULL_TIME,USD,55000.0,45202.0
2,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,140000.0,Full-time,2024-04-12 04:23:32,2024-05-12 04:23:32,NaN,2024-04-12 04:23:32,FULL_TIME,USD,157500.0,11040.0
3,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,300000.0,YEARLY,United States,81942316.0,7.0,60000.0,Contract,2024-04-11 18:43:39,2024-05-11 18:43:39,NaN,2024-04-11 18:43:39,CONTRACT,USD,180000.0,NaN
4,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,HOURLY,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,2024-04-11 17:14:46,2024-05-11 17:14:46,NaN,2024-04-11 17:14:46,FULL_TIME,USD,52000.0,44102.0


In [4]:
df.shape

(32556, 19)

## Further cleaning up and correct salary columns.

In [6]:
#deleting unusal records
df = df[~((df['pay_period'] == 'HOURLY') & (df['max_salary'] > 1000)&(df['max_salary'] < 10000))]

In [7]:
df.shape

(32555, 19)

In [8]:
# correct pay_period types and correct normalized salary
df.loc[(df['pay_period'] == "HOURLY") & (df['max_salary'] >= 500), 'normalized_salary'] = (df['max_salary']+df['min_salary'])/2
df.loc[(df['pay_period'] == "HOURLY") & (df['max_salary'] >= 500), 'pay_period'] = "YEARLY"

In [9]:
#hourly=df[df['pay_period']=='HOURLY']
#hourly['normalized_salary'].describe()

In [10]:
#print(hourly['max_salary'].describe())
#print(hourly['min_salary'].describe())

In [11]:
#convert hourly salary into yearly, 2028 hours for a year
df.loc[df['pay_period'] == "HOURLY", 'max_salary'] *= 2080
df.loc[df['pay_period'] == "HOURLY", 'min_salary'] *= 2080

In [12]:
#remove pay period column since all data are convert to yearly salary
df=df.drop('pay_period',axis=1)
df.shape

(32555, 18)

## Add company info for each postings



In [14]:
company= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/archive/companies/companies.csv',header=0)
industry= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/archive/companies/company_industries.csv',header=0)
specs= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/archive/companies/company_specialities.csv',header=0)
size= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/archive/companies/employee_counts.csv',header=0)

In [15]:
company_full = pd.merge(company, industry, how='outer', on='company_id')

In [16]:
#df['description'][6]

In [17]:
# remove duplication for size records
size=size.loc[size.groupby('company_id')['time_recorded'].idxmax()]
size.head()

#pivot specialty 
specs=specs.groupby('company_id', as_index=False).agg({
    'speciality': ', '.join
})

In [18]:
#merging all sheet
company_full = pd.merge(company_full, specs, how='outer', on='company_id')
company_full = pd.merge(company_full, size, how='outer', on='company_id')

In [19]:
#rename columns to distinguish between job location and headquarter location 
company_full=company_full.rename(columns={'description':'company desc',
                                          'state':'headquarter state',
                                          'city':'headquarter city',
                                          'zip_code':'headquarter zipcode',
                                          'address':'headquarter addr'})

In [20]:
company_full['time_recorded'] = pd.to_datetime(company_full['time_recorded'], unit='ms')

In [53]:
#company_full=company_full.drop(['url'],axis=1)
# Remove duplicate rows based on 'company_id'
company_full = company_full.drop_duplicates(subset='company_id', keep='first')

In [55]:
print(company_full.shape)
print(company_full.columns)

(24473, 14)
Index(['company_id', 'name', 'company desc', 'company_size',
       'headquarter state', 'country', 'headquarter city',
       'headquarter zipcode', 'headquarter addr', 'industry', 'speciality',
       'employee_count', 'follower_count', 'time_recorded'],
      dtype='object')


In [57]:
new_df=df

In [59]:
new_df= pd.merge(new_df,company_full, how='left', on='company_id') 
# Join matching rows from company_full to df.

In [61]:
new_df.shape

(32555, 31)

In [63]:
new_df.head()

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,headquarter state,country,headquarter city,headquarter zipcode,headquarter addr,industry,speciality,employee_count,follower_count,time_recorded
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,41600.0,"Princeton, NJ",2774458.0,20.0,35360.0,Full-time,...,NJ,US,Jersey City,07302,242 Tenth Street,Real Estate,"real estate, new development",402,2351,1970-01-20 19:56:49.275
1,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,"Cincinnati, OH",64896719.0,8.0,45000.0,Full-time,...,Ohio,US,Mariemont,45227,6880 Wooster Pike,Restaurants,NaN,15,40,1970-01-20 19:54:39.321
2,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,"New Hyde Park, NY",766262.0,16.0,140000.0,Full-time,...,New York,US,Lake Success,11042,3 Dakota Drive,Law Practice,"Civil Litigation, Corporate & Securities Law, ...",222,2427,1970-01-20 19:48:17.299
3,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,300000.0,United States,81942316.0,7.0,60000.0,Contract,...,CA,US,Los Angeles,90017,0,Design Services,NaN,11,447,1970-01-20 19:47:41.202
4,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,...,Ohio,US,Cleveland,44114,3635 PERKINS AVE,Non-profit Organizations,NaN,6,289,1970-01-20 19:47:36.403


In [65]:
#new_df.to_csv('posting_32556_salary_info& company info.csv', index=False) 